In [1]:
# python libraries
import numpy as np
from IPython.display import HTML, display

# pydrake imports
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         LinearQuadraticRegulator, Parser,
                         PlanarSceneGraphVisualizer, Simulator, Linearize)

## Problem Description
URDF (Unified Robot Description Format) is one of the most widely used formats to describe the geometry of robots. They are represented in XML and can be stored in strings as we will do later. In this problem, we will build a double pendulum cartpole using the single pendulum cartpole described in [Section 3.2](https://underactuated.csail.mit.edu/acrobot.html#cart_pole) of the textbook as a base to understand the construction of a basic URDF then modify the system into a double pendulum cartpole. We will then wire up an LQR controller and simulate the cart-pole and perform a new series of balancing tasks.

The simple URDFs we will be make today consist of three major components:
1.   **Links**: inertial and visual information for each link
2.   **Joints**: the connection between links
3.   **Transmissions**: control inputs to joints

We will go over each of these three in detail to construct the single pendulum cart-pole from the text with a slight twist.

A link component here has three parts: the name (used to identify the link), the inertial (used to define the mass and  center of mass of the link), and the visual (used for displaying representative images). 

**Note**: For the sake of this problem, we consider x to be the horizontal direction and z to be the vertical direction. 

Below you will find the first link, which will represent the base cart. You can see we define it to have a mass of 1. and a COM of (0., 0., -1.). The visuals consist of a box for the body of the cart and two spheres for the wheels with their positions set relative to the link's origin.


In [2]:
# DO NOT MODIFY
base_urdf = """
  <link name="base">
    <inertial>
      <origin xyz="0 0 0" />
      <mass value="1" />
    </inertial>

    <visual>
      <origin xyz="0 0 0" />
      <geometry>
        <box size=".5 .2 .2" />
      </geometry>
      <material>
        <color rgba="0 1 0 1" />
      </material>
    </visual>

    <visual>
      <origin xyz=".15 0 -.15" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>
    
    <visual>
      <origin xyz="-.15 0 -.15" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>
  </link>
"""

Next we have a templated pendulum link. The pendulum is defined by a ball mass and a cylinder rod where the COM is at the ball mass. We will replace the placeholder text using a helper function later to programmatically add these links. It is worth noting now that origin of a link is relative to the origin of it's parent. So for the first link, the parent origin is the center of the cart.

In [3]:
# DO NOT MODIFY
pendulum_link = """ 
  <link name="pendulumNUMBER">
    <inertial>
      <origin xyz="0 0 REPLACE" />
      <mass value="1" />
    </inertial>

    <visual>
      <origin xyz="0 0 REPLACE" />
      <geometry>
         <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 REPLACE" />
      <geometry>
         <cylinder length="1" radius=".01" />
      </geometry>
      <material>
        <color rgba="1 0 0 1" />
      </material>
    </visual>
  </link>
"""

Now we move onto joints, which are used to connect links and define their behavior. The first joint we must consider is how the cart fits into the world, which is the root parent of our cart-pole system. Here we treat this as a sliding or prismatic joint as if the cart is moving on a fixed track in the x direction. 

In [4]:
# DO NOT MODIFY
base_joint = """
  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="base" />
    <axis xyz="1 0 0" />
  </joint>
"""

The second joint to consider is for pendulum links connected to the cart base as well as other pendula. We will treat these as continuous joints allowing them to revolve around their parent origin. We will again programmatically replace the placeholder text later when we build the full URDF string.

In [5]:
# DO NOT MODIFY
pendulum_joint = """
  <joint name="thetaNUMBER" type="continuous">
    <parent link="PARENT" />
    <child link="CHILD" />
    <axis xyz="0 -1 0" />
    <origin xyz="0 0 REPLACE"/>
  </joint>
"""

Lastly, we come to the transmission component. Here the only controlled input is a force applied in the x direction on the cart base. 

In [6]:
# DO NOT MODIFY
transmission = """
  <transmission type="SimpleTransmission" name="base_force">
    <actuator name="force" />
    <joint name="x" />
  </transmission>
"""

Now we have all the components necessary to construct a URDF for the cart-pole. Below you will find a helper function we will use to do so programmatically. The function takes as input a list of parameters for each pendulum link. For our first test, we will only pass a single list to construct a single pendulum.

Each list contains two paramters: the first is the vertical location of the COM of the ball mass of the pendulum (positive or negative) and the second is origin point for the joint connecting the pendulum to its parent link. 

For example we will construct and run LQR on the cart-pole as defined in the text, where the ball mass is located -1. below the cart (so that $\theta = 0 $ is the stable equilibrium below the cart $\theta = \pi$ is the unstable equilibrium above. The joint origin is thus 0 as the base of the pendula is at the center of the cart.

In [7]:
# function that builds a urdf string with the variable number of pendula
# input is a list for the number of pendula consisting each of a list
# of parameters for the vertical distance from the parent origin
# and the joint origin to the parent
def create_cartpole_urdf(parameters):
  
  # urdf header and ender
  urdf = """<?xml version="1.0"?>\n<robot name="DoubleCartPole">BASE</robot>"""
  # cart base of the robot
  urdf = urdf.replace("BASE", base_urdf + "LINK")
  
  # add pendula links
  for number in range(0,len(parameters)):
    urdf = urdf.replace("LINK", pendulum_link.replace("NUMBER", str(number)).replace("REPLACE", str(parameters[number][0]), 2).replace("REPLACE", str(parameters[number][0]*.5)) + "LINK")
  urdf = urdf.replace("LINK", base_joint + "JOINT")

  # add the joints starting with base
  parent = "base"
  for number in range(0,len(parameters)):
    urdf = urdf.replace("JOINT", pendulum_joint.replace("NUMBER", str(number)).replace("PARENT", parent).replace("CHILD", "pendulum"+str(number)).replace("REPLACE", str(parameters[number][1])) + "JOINT")
    parent = "pendulum" + str(number)

  # add the transmission for the cart base
  urdf = urdf.replace("JOINT", transmission)

  return urdf

For the single pendulum cart-pole we define the unstable equilibrium to be $\theta = \pi$ and $Q$ and $R$ to be identity matrices. We also define the URDF parameters to be **[link_COM_height, joint_origin_height]** which for the single pendulum will be **-1.** and **0.** respectively.

In [16]:
# unstable equilibrium point
x_star = [0, np.pi, 0, 0]

# weight matrices for the lqr controller*
Q = np.diag((10., 10., 1., 1.))
R = np.eye(1)

# construct the parameters for a URDF of single pendulum cart-pole
def get_single_pendulum_urdf_parameters():
  return [[-1., 0.]] # modify here

single_pendulum_cartpole_urdf = create_cartpole_urdf(get_single_pendulum_urdf_parameters())

# verify the urdf file output
print(single_pendulum_cartpole_urdf)

<?xml version="1.0"?>
<robot name="DoubleCartPole">
  <link name="base">
    <inertial>
      <origin xyz="0 0 0" />
      <mass value="1" />
    </inertial>

    <visual>
      <origin xyz="0 0 0" />
      <geometry>
        <box size=".5 .2 .2" />
      </geometry>
      <material>
        <color rgba="0 1 0 1" />
      </material>
    </visual>

    <visual>
      <origin xyz=".15 0 -.15" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>
    
    <visual>
      <origin xyz="-.15 0 -.15" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>
  </link>
 
  <link name="pendulum0">
    <inertial>
      <origin xyz="0 0 -1.0" />
      <mass value="1" />
    </inertial>

    <visual>
      <origin xyz="0 0 -1.0" />
      <geometry>
         <sphere radius=".05" />
     

Now that we've successfully constructed a URDF file, we can add it to LQR and run our robot!

In [18]:
# start construction site of our block diagram
builder = DiagramBuilder()

# instantiate the cart-pole and the scene graph
cartpole, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
Parser(cartpole).AddModelFromString(single_pendulum_cartpole_urdf, "urdf")
cartpole.Finalize()

# set the operating point (vertical unstable equilibrium)
context = cartpole.CreateDefaultContext()
context.get_mutable_continuous_state_vector().SetFromVector(x_star)

# fix the input port to zero and get its index for the lqr function
cartpole.get_actuation_input_port().FixValue(context, [0])
input_i = cartpole.get_actuation_input_port().get_index()

# synthesize lqr controller directly from
# the nonlinear system and the operating point
lqr = LinearQuadraticRegulator(cartpole,
                               context,
                               Q,
                               R,
                               input_port_index=input_i)
lqr = builder.AddSystem(lqr)

# the following two lines are not needed here...
output_i = cartpole.get_state_output_port().get_index()
cartpole_lin = Linearize(cartpole,
                         context,
                         input_port_index=input_i,
                         output_port_index=output_i)

# wire cart-pole and lqr
builder.Connect(cartpole.get_state_output_port(), lqr.get_input_port(0))
builder.Connect(lqr.get_output_port(0), cartpole.get_actuation_input_port())

# add a visualizer and wire it
visualizer = builder.AddSystem(
    PlanarSceneGraphVisualizer(scene_graph, xlim=[-3., 3.], ylim=[-.4, 3.], show=False)
)
builder.Connect(scene_graph.get_query_output_port(), visualizer.get_input_port(0))

# finish building the block diagram
diagram = builder.Build()

# instantiate a simulator
simulator = Simulator(diagram)
simulator.set_publish_every_time_step(False) # makes sim faster

In [19]:
# function that given the cart-pole initial state
# and the simulation time, simulates the system
# and produces a video
def simulate_and_animate(x0, sim_time=5):
    
    # start recording the video for the animation of the simulation
    visualizer.start_recording()
    
    # reset initial time and state
    context = simulator.get_mutable_context()
    context.SetTime(0.)
    context.SetContinuousState(x0)
    
    # run sim
    simulator.Initialize()
    simulator.AdvanceTo(sim_time)
    
    # stop video
    visualizer.stop_recording()
    
    # construct animation
    ani = visualizer.get_recording_as_animation()
    
    # display animation below the cell
    display(HTML(ani.to_jshtml()))
    
    # reset to empty video
    visualizer.reset_recording()

In [20]:
# simulate and animate the cart
simulate_and_animate(np.array([0, 0.9*np.pi, 0, 0]))

Now that we've managed to create the URDF for the single pendulum cart-pole, we have all the tools to make a multi-pendulum cart-pole. For this problem, we will construct a double pendulum cart-pole.

Using the same helper function we can create a list of parameter lists for as many pendula as we would like to add. Consider now where the second pendulum will be situated relative to its parent the first pendulum and how its joint will be positioned. Drawing a sketch is a helpful strategy for working your head around how the links and joints will situate with each other.

**Note:** For the first pendulum, $\theta = 0$ is considered to be the stable downward equilibrium because it is defined in the **-z** direction relative to the base cart's origin. For the second pendulum, we will be relative to the first pendulum's reference direction.

However, we will make one crucial change to the single-pendulum cart-pole. Previously we considered $\theta = 0$ to be the downward stable equilibrium, now however we will switch $\theta = 0$ to be the unstable equilibrium so that all pendulum consider that direction to be the same $\theta$. This is because the position of the links are relative to each other, so any successive pendula past the first will have its $\theta = 0$ set to be colinear to the first pendulum. This will nicely clean up our system and challenge you to think about how to change the URDF parameters to make this change. 

All values for both pendula should be {-1., 0., 1.} inclusive.

In [252]:
# unstable equilibrium point
x_star = [0, np.pi, np.pi, 0, 0, 0]

# weight matrices for the lqr controller
Q = np.diag((10., 10., 10., 1., 1., 1.))
R = np.eye(1)

# construct the parameters for a URDF of double pendulum cart-pole
def get_double_pendulum_urdf_parameters():
  return [[1, 0], [1, 1]] # modify here
    # return [[-1, 0], [-1, -1]] # modify here  # downward equilibrium case
double_pendulum_cartpole_urdf = create_cartpole_urdf(get_double_pendulum_urdf_parameters())

# verify the urdf file output
# print(double_pendulum_cartpole_urdf)

In [253]:
builder = DiagramBuilder()

cartpole, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
Parser(cartpole).AddModelFromString(double_pendulum_cartpole_urdf, "urdf")
cartpole.Finalize()

context = cartpole.CreateDefaultContext()
context.get_mutable_continuous_state_vector().SetFromVector(x_star)
cartpole.get_actuation_input_port().FixValue(context, [0])
input_i = cartpole.get_actuation_input_port().get_index()
lqr = LinearQuadraticRegulator(cartpole,
                               context,
                               Q,
                               R,
                               input_port_index=input_i)
lqr = builder.AddSystem(lqr)
output_i = cartpole.get_state_output_port().get_index()
cartpole_lin = Linearize(cartpole,
                         context,
                         input_port_index=input_i,
                         output_port_index=output_i)
builder.Connect(cartpole.get_state_output_port(), lqr.get_input_port(0))
builder.Connect(lqr.get_output_port(0), cartpole.get_actuation_input_port())

visualizer = builder.AddSystem(
    PlanarSceneGraphVisualizer(scene_graph, xlim=[-3., 3.], ylim=[-.4, 3], show=False)
)
builder.Connect(scene_graph.get_query_output_port(), visualizer.get_input_port(0))
diagram = builder.Build()

simulator = Simulator(diagram)
simulator.set_publish_every_time_step(False)
simulator.get_mutable_integrator().set_fixed_step_mode(True)

def simulate_and_animate(x0, sim_time=5):

    # start recording the video for the animation of the simulation
    visualizer.start_recording()

    # reset initial time and state
    context = simulator.get_mutable_context()
    context.SetTime(0.)
    context.SetContinuousState(x0)

    # run sim
    simulator.Initialize()
    simulator.AdvanceTo(sim_time)

    # stop video
    visualizer.stop_recording()

    # construct animation
    ani = visualizer.get_recording_as_animation()

    # display animation below the cell
    display(HTML(ani.to_jshtml()))

    # reset to empty video
    visualizer.reset_recording()

In [254]:
# simulate and animate the cart
simulate_and_animate(np.array([-2, 0.96*np.pi, 0.93*np.pi, 0, 0, 0]))

In [251]:
from underactuated.exercises.acrobot.test_double_cartpole_urdf import TestDoubleCartPoleURDF
from underactuated.exercises.grader import Grader
Grader.grade_output([TestDoubleCartPoleURDF], [locals()], 'results.json')
Grader.print_test_results('results.json')

Total score is 3/8.

Score for Test the single_pendulum_urdf_parameters is 3/3.

Score for Test the double_pendulum_urdf_parameters is 0/5.
- Test Failed: -7115120300195466536 != -4203674514941357763 : Incorrect double pendulum urdf parameters.



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1bc26dfc-d6cd-4ced-9fa0-9d580c5165cd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>